# Keyword Spotting: 「asial」 vs 指示コマンド vs unknown / background_noise（TFJS出力）

このノートブックは、DigiKey のチュートリアルが説明している **Speech Commands + _background_noise_ + 自作キーワード** の考え方（1秒クリップ、16kHz、ノイズ混ぜ、MFCC→CNN）を Colab 上で再現し、最後に **TensorFlow.js (LayersModel) の `model.json`** を出力します。  
参考：DigiKey 記事（Speech Commands の取得、_background_noise_ の分離、16kHz/1秒、MFCC+CNN） citeturn12view0

---

## このノートでやること（ざっくり）
1. Speech Commands をダウンロードして、指示コマンドと unknown を作る  
2. _background_noise_ から 1秒ノイズ片を作り、さらに学習時にランダムにミックス  
3. `asial` の自作音声（あなたの録音）を読み込み  
4. WAV を **16kHz / mono / 1.0秒 / float32** に統一（重要）  
5. MFCC を作って CNN を学習  
6. TFJS 向けに `model.json` を出力

In [ ]:
#@title 0) インストール（Colabの依存衝突を避ける版）
# ポイント:
# - Colab既存の bigquery/xarray 等が packaging>=24 を要求する一方、tensorflowjs が packaging 23.x を要求して衝突しがちです。
# - ここでは tensorflowjs を --no-deps で入れて変換ツールだけ使います（変換自体は通常これで動きます）。
# - DEMAND/LibriSpeech 取り込みを使いたい場合は datasets を追加で入れてください（ただし fsspec の衝突に注意）。

!pip -q install -U pip

# 学習に必要なもの
!pip -q install   "tensorflow==2.19.0"   "tensorflow-decision-forests==1.12.0"   "librosa==0.10.1" "soundfile==0.12.1"   "tqdm>=4.67" "scikit-learn>=1.6"

# TFJS 変換ツール（依存解決はしない）
!pip -q install "tensorflowjs==4.22.0" --no-deps

import sys, tensorflow as tf
print("Python:", sys.version)
print("TensorFlow:", tf.__version__)

import tensorflowjs as tfjs
print("TensorFlow.js (python):", tfjs.__version__)

In [ ]:
#@title （変換でKeras互換性エラーが出る場合のみ）Keras 2互換に切り替え
# TF 2.16+ は既定で Keras 3 になるため、必要なら tf-keras + TF_USE_LEGACY_KERAS で Keras 2 を使えます citeturn5search3
import os
os.environ["TF_USE_LEGACY_KERAS"] = "1"  # TensorFlow import より前に設定する必要があります citeturn5search3
!pip -q install "tf-keras~=2.19"
print("Set TF_USE_LEGACY_KERAS=1 and installed tf-keras.")

## 1) 設定（ラベルなど）
- 指示コマンド（例）：`up, down, left, right, go, stop`  
- 自作キーワード：`asial`  
- 追加クラス：`unknown`, `background_noise`

※ Speech Commands recognizer の既定語彙にも `unknown` と `background_noise` が含まれます citeturn7search5

In [ ]:
import os, random, glob, shutil, math
from pathlib import Path

SEED = 42
random.seed(SEED)

# ---- ここを必要に応じて変更 ----
COMMAND_WORDS = ["up", "down", "left", "right", "go", "stop"]   # 指示コマンド
CUSTOM_WORD   = "asial"                      # 自作キーワード（フォルダ名）
LABELS = COMMAND_WORDS + [CUSTOM_WORD, "unknown", "background_noise"]

SR = 16000
CLIP_SECONDS = 1.0
CLIP_SAMPLES = int(SR * CLIP_SECONDS)

# データ配置
WORK = Path("/content/kws")
RAW  = WORK / "raw"      # 収集した（まだバラバラな）wav
STD  = WORK / "std"      # 16kHz/mono/1s に統一した wav
NOISE_POOL = WORK / "noise_pool"  # ミックス用ノイズ片
for p in [RAW, STD, NOISE_POOL]:
    p.mkdir(parents=True, exist_ok=True)

print("LABELS:", LABELS)
print("WORK:", WORK)

## 2) Speech Commands を取得して、指示コマンド / unknown / background_noise を作る

DigiKey 記事でも、Speech Commands をダウンロードして `_background_noise_` を分離して使う手順が説明されています citeturn12view0  
Speech Commands は CC BY 4.0 で公開されています（引用元例） citeturn0search5

In [ ]:
#@title 2-1) Speech Commands v0.02 をダウンロードして展開
import tarfile, urllib.request

speech_tar = WORK / "speech_commands_v0.02.tar.gz"
speech_root = WORK / "speech_commands_v0.02"

if not speech_root.exists():
    if not speech_tar.exists():
        url = "https://storage.googleapis.com/download.tensorflow.org/data/speech_commands_v0.02.tar.gz"
        print("Downloading:", url)
        urllib.request.urlretrieve(url, speech_tar)
    print("Extracting...")
    speech_root.mkdir(parents=True, exist_ok=True)
    with tarfile.open(speech_tar, "r:gz") as tar:
        tar.extractall(path=speech_root)
print("speech_root:", speech_root)
print("Example dirs:", sorted([p.name for p in speech_root.iterdir() if p.is_dir()])[:10])

In [ ]:
#@title 2-2) 指示コマンド WAV を RAW にコピー
def copy_some(src_dir: Path, dst_dir: Path, max_files=None):
    dst_dir.mkdir(parents=True, exist_ok=True)
    files = sorted(src_dir.glob("*.wav"))
    if max_files is not None:
        random.shuffle(files)
        files = files[:max_files]
    for f in files:
        shutil.copy2(f, dst_dir / f.name)
    return len(files)

# 指示コマンド：各フォルダを丸ごとコピー（必要なら max_files で制限）
for w in COMMAND_WORDS:
    n = copy_some(speech_root / w, RAW / w, max_files=None)
    print(w, n)

# background_noise の元（長い wav）
bg_src = speech_root / "_background_noise_"
print("background_noise sources:", len(list(bg_src.glob("*.wav"))), bg_src)

In [ ]:
#@title 2-3) unknown を作る：Speech Commands の「それ以外の単語」からランダム抽出
# unknown の比率は調整ポイント。とりあえず各コマンド合計と同程度に作る。
target_unknown = sum(len(list((RAW / w).glob("*.wav"))) for w in COMMAND_WORDS)

exclude = set(COMMAND_WORDS + [CUSTOM_WORD, "_background_noise_", "unknown", "background_noise"])
candidate_dirs = [p for p in speech_root.iterdir() if p.is_dir() and p.name not in exclude and not p.name.startswith(".")]

cand_files = []
for d in candidate_dirs:
    cand_files.extend(list(d.glob("*.wav")))
random.shuffle(cand_files)
cand_files = cand_files[:target_unknown]

dst = RAW / "unknown"
dst.mkdir(parents=True, exist_ok=True)
for f in cand_files:
    shutil.copy2(f, dst / f"{f.parent.name}_{f.name}")

print("unknown:", len(list(dst.glob("*.wav"))), "target:", target_unknown)

## 3) background_noise クラス用の 1秒ノイズ片を作る（+ ノイズプール）

DigiKey 記事でも、`_background_noise_` の長尺 WAV を使ってノイズを混ぜる（データ拡張）流れが説明されています citeturn12view0  
ここでは
- **background_noise（分類クラス）用**：純ノイズ 1秒片を大量に作る  
- **noise_pool（ミックス用）**：同じものを流用  
を行います。

In [ ]:
#@title 3) _background_noise_ から 1秒ノイズ片を生成
import numpy as np
import soundfile as sf
import librosa
from tqdm import tqdm

def ensure_mono(x):
    if x.ndim == 1:
        return x
    return np.mean(x, axis=1)

def pad_or_trim(x, n):
    if len(x) < n:
        return np.pad(x, (0, n - len(x)))
    return x[:n]

def slice_noise_file(wav_path: Path, out_dir: Path, n_clips: int, prefix: str):
    y, sr = librosa.load(str(wav_path), sr=SR, mono=True)
    if len(y) < CLIP_SAMPLES:
        return 0
    out_dir.mkdir(parents=True, exist_ok=True)
    count = 0
    max_start = len(y) - CLIP_SAMPLES
    for i in range(n_clips):
        start = random.randint(0, max_start)
        clip = y[start:start+CLIP_SAMPLES]
        out = out_dir / f"{prefix}_{wav_path.stem}_{i:04d}.wav"
        sf.write(out, clip.astype(np.float32), SR, subtype="PCM_16")
        count += 1
    return count

bg_dst = RAW / "background_noise"
bg_dst.mkdir(parents=True, exist_ok=True)

# 各ノイズファイルから生成する数（必要に応じて増減）
CLIPS_PER_BG_FILE = 400

total = 0
for f in tqdm(sorted(bg_src.glob("*.wav"))):
    total += slice_noise_file(f, bg_dst, CLIPS_PER_BG_FILE, prefix="bg")
print("background_noise clips:", total)

# noise_pool にもコピー（ミックス用に使う）
NOISE_POOL.mkdir(parents=True, exist_ok=True)
for f in bg_dst.glob("*.wav"):
    shutil.copy2(f, NOISE_POOL / f.name)

print("noise_pool clips:", len(list(NOISE_POOL.glob('*.wav'))))

## 4) 追加のオープンソースデータセット（任意）

「unknown」や「background_noise」をより多様にするなら、Speech Commands 以外も混ぜるのが有効です。

### 4-A) DEMAND（環境ノイズ）
- 実環境のノイズ録音が入っているデータセット（Hugging Face でも入手例あり） citeturn1search0turn1search8  
- ライセンスは CC BY-SA 3.0 として提供されている配布元があります citeturn1search16

### 4-B) MUSAN（noise/music/speech）
- ノイズ・音楽・音声を含むコーパス。論文では「柔軟な Creative Commons ライセンス」で公開と説明されています citeturn0search10turn0search2

### 4-C) LibriSpeech（unknown の“人の声”を増やす）
- 大規模な朗読音声。CC BY 4.0 citeturn0search3

### 4-D) Mozilla Common Voice（unknown を多言語で増やす）
- CC0 でデータセット配布（ただし配布経路の注意が書かれています） citeturn1search7turn1search3

このノートでは **軽量に試せる DEMAND と LibriSpeech の “小さめ” 手順** を用意します（実行は任意）。

In [ ]:
#@title 4-A) （任意）DEMAND ノイズを noise_pool に追加（短いデモ）
# 実行すると、Hugging Face から DEMAND の音声を取得して 1秒ノイズ片を追加します。
# 環境ノイズの多様性が上がります。

USE_DEMAND = False  #@param {type:"boolean"}

if USE_DEMAND:
    from datasets import load_dataset
    import numpy as np
    import soundfile as sf

    ds = load_dataset("voice-biomarkers/DEMAND-acoustic-noise", split="train")
    out_dir = NOISE_POOL / "demand"
    out_dir.mkdir(parents=True, exist_ok=True)

    clips_per_item = 40
    total = 0
    for i in range(min(len(ds), 15)):
        arr = ds[i]["audio"]["array"]
        sr0 = ds[i]["audio"]["sampling_rate"]
        y = librosa.resample(arr.astype(np.float32), orig_sr=sr0, target_sr=SR)
        if len(y) < CLIP_SAMPLES:
            continue
        max_start = len(y) - CLIP_SAMPLES
        for j in range(clips_per_item):
            start = random.randint(0, max_start)
            clip = y[start:start+CLIP_SAMPLES]
            sf.write(out_dir / f"demand_{i:02d}_{j:03d}.wav", clip.astype(np.float32), SR, subtype="PCM_16")
            total += 1
    print("Added DEMAND noise clips:", total)
    print("noise_pool total:", len(list(NOISE_POOL.rglob('*.wav'))))
else:
    print("Skipped DEMAND.")

In [ ]:
#@title 4-C) （任意）LibriSpeech から unknown（人の声）を追加（dev-clean の一部だけ）
USE_LIBRISPEECH = False  #@param {type:"boolean"}

if USE_LIBRISPEECH:
    import tarfile, urllib.request
    lib_root = WORK / "librispeech_dev_clean"
    tgz = WORK / "dev-clean.tar.gz"
    if not lib_root.exists():
        url = "https://www.openslr.org/resources/12/dev-clean.tar.gz"
        print("Downloading:", url)
        urllib.request.urlretrieve(url, tgz)
        print("Extracting...")
        lib_root.mkdir(parents=True, exist_ok=True)
        with tarfile.open(tgz, "r:gz") as tar:
            tar.extractall(path=lib_root)

    # flac -> wav 変換せず、librosa で直接読み込んで 1秒切り出し（軽量）
    flacs = list(lib_root.rglob("*.flac"))
    random.shuffle(flacs)
    flacs = flacs[:60]  # 取りすぎ防止

    out_dir = RAW / "unknown_librispeech"
    out_dir.mkdir(parents=True, exist_ok=True)

    total = 0
    for f in tqdm(flacs):
        y, sr0 = librosa.load(str(f), sr=SR, mono=True)
        if len(y) < CLIP_SAMPLES:
            continue
        max_start = len(y) - CLIP_SAMPLES
        # 1ファイルから数枚だけ
        for j in range(3):
            start = random.randint(0, max_start)
            clip = y[start:start+CLIP_SAMPLES]
            sf.write(out_dir / f"ls_{f.stem}_{j}.wav", clip.astype(np.float32), SR, subtype="PCM_16")
            total += 1

    print("Added LibriSpeech unknown clips:", total)

    # unknown にマージ
    unk_dir = RAW / "unknown"
    for f in out_dir.glob("*.wav"):
        shutil.copy2(f, unk_dir / f.name)

    print("unknown total:", len(list(unk_dir.glob('*.wav'))))
else:
    print("Skipped LibriSpeech.")

## 5) `asial` の自作データを用意（アップロード or Drive）
以下どちらかで `asial` の WAV を `RAW/asial/` に置きます。

- A) Colab に ZIP をアップロード（中に wav が入っていればOK）
- B) Google Drive からコピー

※ DigiKey 記事では、カスタムキーワードは「最低50サンプル、1秒、16kHz、WAV」を推奨しています citeturn12view0

In [ ]:
#@title 5-A) Colab にZIPをアップロード（任意）
from google.colab import files

UPLOAD_ZIP = False  #@param {type:"boolean"}
if UPLOAD_ZIP:
    uploaded = files.upload()
    # 1つ目のzipを使う想定
    zip_path = next(iter(uploaded.keys()))
    print("Uploaded:", zip_path)

    import zipfile
    asial_dir = RAW / CUSTOM_WORD
    asial_dir.mkdir(parents=True, exist_ok=True)

    with zipfile.ZipFile(zip_path, "r") as z:
        z.extractall(asial_dir)

    # zip内に階層がある場合を吸収（wavを集約）
    wavs = list(asial_dir.rglob("*.wav"))
    flat_dir = asial_dir
    for w in wavs:
        if w.parent != flat_dir:
            shutil.copy2(w, flat_dir / w.name)

    print("asial wav count:", len(list((RAW / CUSTOM_WORD).glob('*.wav'))))
else:
    print("Skip upload.")

In [ ]:
#@title 5-B) Google Drive からコピー（任意）
USE_DRIVE = False  #@param {type:"boolean"}
DRIVE_ASIAL_DIR = "/content/drive/MyDrive/asial_wavs"  #@param {type:"string"}

if USE_DRIVE:
    from google.colab import drive
    drive.mount("/content/drive")

    src = Path(DRIVE_ASIAL_DIR)
    assert src.exists(), f"not found: {src}"
    dst = RAW / CUSTOM_WORD
    dst.mkdir(parents=True, exist_ok=True)

    for f in src.glob("*.wav"):
        shutil.copy2(f, dst / f.name)
    print("asial wav count:", len(list(dst.glob('*.wav'))))
else:
    print("Skip drive.")

## 6) WAV を統一（16kHz / mono / 1.0秒 / PCM16（16-bit PCM））

ここが「一番ハマる」ポイントです（形式が揃ってないと学習が壊れやすい）。  
DigiKey 記事でも 16kHz・1秒・WAV を強調しています citeturn12view0

In [ ]:
#@title 6) 全ラベルの wav を標準化して STD に出力
import numpy as np
import librosa
import soundfile as sf
from tqdm import tqdm

def standardize_wav(in_path: Path, out_path: Path):
    y, sr0 = librosa.load(str(in_path), sr=SR, mono=True)  # resample + mono
    y = pad_or_trim(y.astype(np.float32), CLIP_SAMPLES)
    out_path.parent.mkdir(parents=True, exist_ok=True)
    sf.write(str(out_path), y, SR, subtype="PCM_16")

# まず STD を空にする
if STD.exists():
    shutil.rmtree(STD)
STD.mkdir(parents=True, exist_ok=True)

# RAW の各ラベルから STD へ
for label in LABELS:
    src_dir = RAW / label
    if not src_dir.exists():
        print("WARN: missing label dir:", src_dir)
        continue
    files = sorted(src_dir.glob("*.wav"))
    for f in tqdm(files, desc=f"std:{label}"):
        out = STD / label / f.name
        standardize_wav(f, out)

for label in LABELS:
    print(label, len(list((STD/label).glob("*.wav"))))

## 7) 学習用 Dataset（tf.data）を作る
- 学習時に **noise_pool からランダムにノイズを混ぜる**（SNRランダム）  
- 特徴量は **MFCC**（DigiKey 記事が採用している特徴量） citeturn12view0

In [ ]:
import tensorflow as tf
import numpy as np

AUTOTUNE = tf.data.AUTOTUNE

label_to_id = {l:i for i,l in enumerate(LABELS)}
id_to_label = {i:l for l,i in label_to_id.items()}

# noise pool files
noise_files = [str(p) for p in NOISE_POOL.rglob("*.wav")]
print("noise_files:", len(noise_files))

def load_wav(path):
    audio = tf.io.read_file(path)
    wav, sr = tf.audio.decode_wav(audio, desired_channels=1)
    wav = tf.squeeze(wav, axis=-1)  # [samples]
    wav = wav[:CLIP_SAMPLES]
    wav = tf.cond(tf.shape(wav)[0] < CLIP_SAMPLES,
                  lambda: tf.pad(wav, [[0, CLIP_SAMPLES - tf.shape(wav)[0]]]),
                  lambda: wav)
    return wav

def random_mix_noise(wav):
    if len(noise_files) == 0:
        return wav
    nf = tf.random.uniform([], 0, len(noise_files), dtype=tf.int32)
    noise = load_wav(tf.constant(noise_files)[nf])

    # SNR: 0〜20dB（調整可）
    snr_db = tf.random.uniform([], 0.0, 20.0)
    wav_rms = tf.sqrt(tf.reduce_mean(tf.square(wav)) + 1e-9)
    noi_rms = tf.sqrt(tf.reduce_mean(tf.square(noise)) + 1e-9)

    snr = tf.pow(10.0, snr_db / 20.0)
    scale = wav_rms / (snr * noi_rms + 1e-9)
    mixed = wav + noise * scale

    # クリップ防止
    mixed = tf.clip_by_value(mixed, -1.0, 1.0)
    return mixed

def wav_to_mfcc(wav):
    # STFT
    frame_length = 640   # 40ms @16k
    frame_step   = 320   # 20ms
    fft_length   = 1024
    stft = tf.signal.stft(wav, frame_length=frame_length, frame_step=frame_step, fft_length=fft_length)
    spectrogram = tf.abs(stft)

    # mel
    num_spectrogram_bins = spectrogram.shape[-1]
    num_mel_bins = 40
    lower_edge_hertz, upper_edge_hertz = 20.0, SR/2
    mel_w = tf.signal.linear_to_mel_weight_matrix(
        num_mel_bins, num_spectrogram_bins, SR, lower_edge_hertz, upper_edge_hertz)
    mel = tf.tensordot(spectrogram, mel_w, 1)
    mel.set_shape(spectrogram.shape[:-1].concatenate(mel_w.shape[-1:]))

    log_mel = tf.math.log(mel + 1e-6)
    mfcc = tf.signal.mfccs_from_log_mel_spectrograms(log_mel)[..., :13]  # 13次元
    # CNN用に [T, F, 1]
    mfcc = mfcc[..., tf.newaxis]
    return mfcc

def make_file_label_lists():
    paths = []
    labels = []
    for label in LABELS:
        for f in (STD / label).glob("*.wav"):
            paths.append(str(f))
            labels.append(label_to_id[label])
    # shuffle
    idx = list(range(len(paths)))
    random.shuffle(idx)
    paths = [paths[i] for i in idx]
    labels = [labels[i] for i in idx]
    return paths, labels

paths, labels = make_file_label_lists()
print("total files:", len(paths))

# train/val split
split = int(0.85 * len(paths))
train_paths, val_paths = paths[:split], paths[split:]
train_labels, val_labels = labels[:split], labels[split:]

def build_ds(paths, labels, training: bool):
    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    if training:
        ds = ds.shuffle(2048, seed=SEED, reshuffle_each_iteration=True)
    def _map(p, y):
        wav = load_wav(p)
        if training:
            wav = random_mix_noise(wav)
        x = wav_to_mfcc(wav)
        return x, tf.one_hot(y, depth=len(LABELS))
    ds = ds.map(_map, num_parallel_calls=AUTOTUNE)
    ds = ds.batch(64).prefetch(AUTOTUNE)
    return ds

train_ds = build_ds(train_paths, train_labels, True)
val_ds   = build_ds(val_paths, val_labels, False)

# shape check
for x,y in train_ds.take(1):
    print("x:", x.shape, "y:", y.shape)

## 8) モデル（CNN）を学習

DigiKey 記事では Edge Impulse の “Audio (MFCC) + Neural Network (Keras)” を使ってキーワード分類を行っています citeturn12view0  
ここでは同じ考え方で、MFCC を画像っぽく扱う CNN を学習します。

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

# 入力形状を ds から取る
for xb, yb in train_ds.take(1):
    input_shape = xb.shape[1:]
print("input_shape:", input_shape)

model = keras.Sequential([
    layers.Input(shape=input_shape),
    layers.Conv2D(16, (3,3), activation="relu", padding="same"),
    layers.MaxPool2D((2,2)),
    layers.Conv2D(32, (3,3), activation="relu", padding="same"),
    layers.MaxPool2D((2,2)),
    layers.Dropout(0.25),
    layers.Conv2D(64, (3,3), activation="relu", padding="same"),
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.25),
    layers.Dense(len(LABELS), activation="softmax")
])

model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=6, restore_best_weights=True)
]

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=40,
    callbacks=callbacks
)

## 9) 評価（混同行列）

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

# 予測
y_true = []
y_pred = []
for xb, yb in val_ds:
    yp = model.predict(xb, verbose=0)
    y_true.extend(np.argmax(yb.numpy(), axis=1))
    y_pred.extend(np.argmax(yp, axis=1))

cm = confusion_matrix(y_true, y_pred)
print("Confusion matrix (rows=true, cols=pred):")
print(cm)
print()
print(classification_report(y_true, y_pred, target_names=LABELS))

## 10) TFJS（model.json）に変換して出力

このノートは Keras モデルを **TensorFlow.js LayersModel** に変換して `model.json` を生成します。  
TensorFlow.js の Speech Commands 系のカスタム学習も「Keras→TFJS 変換」を前提にしています citeturn2search2turn2search1

In [ ]:
import tensorflow as tf
import tensorflowjs as tfjs
import zipfile
import json

# --- TFJS出力先（ブラウザ側のHTMLと合わせる） ---
export_dir = WORK / "tfjs_model_tfonly"
if export_dir.exists():
    shutil.rmtree(export_dir)
export_dir.mkdir(parents=True, exist_ok=True)

# --- モデル保存 ---
tfjs.converters.save_keras_model(model, str(export_dir))
print("Saved TFJS model to:", export_dir)
print("Files:", [p.name for p in export_dir.iterdir()])

# --- 前処理パラメータ（JS側と一致させる） ---
FRAME_LENGTH = 640  # 40ms @ 16k
FRAME_STEP   = 320  # 20ms
FFT_LENGTH   = 1024

NUM_MEL_BINS = 40
LOWER_EDGE_HZ, UPPER_EDGE_HZ = 20.0, SR/2

NUM_MFCC = 13
NUM_SPECTROGRAM_BINS = FFT_LENGTH // 2 + 1

# ラベル + 前処理情報（JS側で読めるように）
with open(export_dir / "labels.json", "w", encoding="utf-8") as f:
    json.dump({
        "labels": LABELS,
        "sample_rate": int(SR),
        "clip_samples": int(CLIP_SAMPLES),
        "frame_length": int(FRAME_LENGTH),
        "frame_step": int(FRAME_STEP),
        "fft_length": int(FFT_LENGTH),
        "num_mel_bins": int(NUM_MEL_BINS),
        "lower_edge_hz": float(LOWER_EDGE_HZ),
        "upper_edge_hz": float(UPPER_EDGE_HZ),
        "num_mfcc": int(NUM_MFCC),
        "num_spectrogram_bins": int(NUM_SPECTROGRAM_BINS)
    }, f, ensure_ascii=False, indent=2)

# --- 方式B: 学習側(TF)と同じ行列を吐き出してJSで使う ---
# 1) Mel重み行列: shape [num_spectrogram_bins, num_mel_bins]
mel_w = tf.signal.linear_to_mel_weight_matrix(
    NUM_MEL_BINS, NUM_SPECTROGRAM_BINS, SR, LOWER_EDGE_HZ, UPPER_EDGE_HZ
)
mel_w_np = mel_w.numpy().astype("float32")

with open(export_dir / "mel_weight_matrix.json", "w", encoding="utf-8") as f:
    json.dump({
        "shape": list(mel_w_np.shape),
        "data": mel_w_np.flatten().tolist()
    }, f)

# 2) DCT行列: tf.signal.mfccs_from_log_mel_spectrograms 相当
#    mfccs_from_log_mel_spectrograms は log-mel に DCT(type=2, norm='ortho') をかけて先頭NUM_MFCCを取る
dct_full = tf.signal.dct(tf.eye(NUM_MEL_BINS), type=2, norm='ortho')  # [M, M]
dct_m = dct_full[:, :NUM_MFCC]                                        # [M, NUM_MFCC]
dct_m_np = dct_m.numpy().astype("float32")

with open(export_dir / "dct_matrix.json", "w", encoding="utf-8") as f:
    json.dump({
        "shape": list(dct_m_np.shape),
        "data": dct_m_np.flatten().tolist()
    }, f)

print("Saved feature matrices:",
      (export_dir / "mel_weight_matrix.json").name,
      (export_dir / "dct_matrix.json").name)

# zip 化してダウンロード
zip_path = WORK / "tfjs_model_tfonly.zip"
with zipfile.ZipFile(zip_path, "w", compression=zipfile.ZIP_DEFLATED) as z:
    for p in export_dir.iterdir():
        z.write(p, arcname=p.name)

print("ZIP:", zip_path)


In [ ]:
#@title 10-B) 生成物をダウンロード
from google.colab import files
files.download(str(WORK / "tfjs_model_tfonly.zip"))


## 11) （参考）ブラウザ側で使うときの最低限の考え方
- `model.json` をホスティングして `tf.loadLayersModel()` で読む
- マイク入力を **16kHz / 1秒** に揃える
- Python 側と同じ前処理（STFT→mel→log→MFCC）を TFJS でも行う

`speech-commands` ライブラリは、Speech Commands 系の音声認識をブラウザで扱うためのモジュールで、既定語彙に `unknown` と `background_noise` を含みます citeturn7search5turn7search7  
ただし、このノートのモデルは **MFCC を自前計算**する前提なので、JS側でも同じ特徴量生成を実装して推論してください（実装例のセルは必要なら次で追加できます）。